In [2]:
import pandas as pd
import numpy as np

- 机器学习在定位上的应用：https://cloud.tencent.com/developer/news/701639

In [3]:
from sklearn.utils import shuffle  

# 加载数据集
def load_data(data_path, shuff=True):
    data = pd.read_csv(data_path)
    data = data.fillna(-1)  # Nan值填充
#     print(data.iloc[1000])
    if shuff:   # 将数据集打乱
        data = shuffle(data)
#     print(data.iloc[1000])
    # 构建数据集和标签
    samples = []
    labels = []
    for index, row in data.iterrows():
    # print(row) # 输出每行的索引值
        matrix = []  # 7 * 4
        for i in range(7):
            matrix.append([i+1,
                           row['AsuLevel_' + str(i+1)],
                           row['SignalLevel_' + str(i+1)],
                           row['Dbm_' + str(i+1)]]) # 新数据集中不存在RSSI，使用Dbm(代功率的绝对值)
        # print(matrix)
        samples.append(matrix)
        labels.append([row['Longitude'], row['Latitude']])
    return np.array(samples), np.array(labels)

In [4]:
path = r"./siping_4g.csv"
samples, labels = load_data(path)
print(samples[0], labels[0])

[[  1.  64.   4. -76.]
 [  2.  66.   4. -74.]
 [  3.  51.   4. -89.]
 [  4.  51.   4. -89.]
 [  5.  -1.  -1.  -1.]
 [  6.  -1.  -1.  -1.]
 [  7.  -1.  -1.  -1.]] [121.4960706   31.28294881]


In [5]:
print(samples.shape)
print(labels.shape)

(3479, 7, 4)
(3479, 2)


In [6]:
# 数据变形： (3479, 7, 4)---》 (3479, 7, 4，1)
samples = samples.reshape(-1, 7,4,1) # -1自动计算

In [7]:
from sklearn.model_selection import train_test_split

# 划分数据集
x_train, x_test, y_train, y_test = train_test_split(samples, labels, test_size=0.2)  #划分训练数据、训练标签、验证数据、验证标签

print(len(x_train))
print(len(y_train))
print(x_train[0])
print(y_train[0])

# print(x_test[100])
# print(y_test[100])
print(y_train.shape)
print(x_train.shape)

2783
2783
[[[   1.]
  [  63.]
  [   4.]
  [ -77.]]

 [[   2.]
  [  45.]
  [   4.]
  [ -95.]]

 [[   3.]
  [  38.]
  [   3.]
  [-102.]]

 [[   4.]
  [  42.]
  [   3.]
  [ -98.]]

 [[   5.]
  [  -1.]
  [  -1.]
  [  -1.]]

 [[   6.]
  [  -1.]
  [  -1.]
  [  -1.]]

 [[   7.]
  [  -1.]
  [  -1.]
  [  -1.]]]
[121.49834     31.28468888]
(2783, 2)
(2783, 7, 4, 1)


In [54]:
# 尝试在竞争对手中的前10%，只需要多走一步

from keras.utils import np_utils # keras中numpy工具包
from keras.models import Sequential
# 二维卷积 数据池化  数据扁平化(reshape)
from keras.layers import Dense, Convolution2D, MaxPooling2D, Flatten, Dropout
# 优化器
# from keras.optimizers import Adam
from keras.optimizers import adam_v2

# # 换为one-hot格式
# y_train = np_utils.to_categorical(y_train, num_classes=122)
# y_test = np_utils.to_categorical(y_test, num_classes=122)
# print(y_train[0,0,:])

In [86]:
# 训练模型
model.fit(x_train, y_train, batch_size=64, epochs=100,validation_split=0.2, verbose=0)

# 评估模型
loss, accuracy = model.evaluate(x_test, y_test)
print('test loss: ', loss)
print('test accuracy: ', accuracy)

loss, accuracy = model.evaluate(x_train, y_train)
print('train loss: ', loss)
print('train accuracy: ', accuracy)

Epoch 1/50
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, 

TypeError: must be real number, not Tensor

In [ ]:

# 定义顺序模型
model = Sequential()

# 第一个卷积层
# input_shape 输入数据的形状(平面)，只要在第一层进行设置
"""
    filters, 卷积核/滤波器的个数
    kernel_size, 卷积核
    strides=(1, 1),  步长
    padding='valid',  填充方式
    data_format=None,
    dilation_rate=(1, 1),
    activation=None,  激活函数
    use_bias=True,
    kernel_initializer='glorot_uniform',
    bias_initializer='zeros',
    kernel_regularizer=None,
    bias_regularizer=None,
    activity_regularizer=None,
    kernel_constraint=None,
    bias_constraint=None,
"""
model.add(
    Convolution2D(
        input_shape = (7, 4,1),
        filters = 32,
        kernel_size = 3,
        strides=1,
        padding='same',
        activation='relu'
    )
)

# 2 7 2

# 第一个池化层:不需要输入形状 28*28 --> 28*28（same填充方式）
model.add(
    MaxPooling2D(
        pool_size=2,
        strides=2,
        padding='same',
    )
)

# 第二个卷积层 池化之后 28/2  28*28--> 14*14
model.add(Convolution2D(64,5, strides=1, padding='same', activation='relu'))

# 第二个池化层  14*14
model.add(MaxPooling2D(2,2,'same'))

# 把第二个池化层的输出扁平化为一维   7*7
model.add(Flatten()) # 7*7  --> 64*7*7

# 第一个全连接层 
model.add(Dense(1024, activation='relu'))

# Dropout防止过拟合
model.add(Dropout(0.5))

# 第2个全连接层
model.add(Dense(units=1,activation="relu"))

# model.add(Dense(1))
# 定义优化器  loss 交叉熵
# adam = Adam(lr=1e-4)
lr = 1e-3
adam = adam_v2.Adam(learning_rate=lr)
# model.compile(optimizer=adam, loss='mape', metrics=['mae'])
import tensorflow as tf
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
def distance(true_pt, pred_pt):
#     true_pt[0]=tf.cast(true_pt[0], tf/.float32)
    print("-----------",type(true_pt))
#     true_pt=true_pt.to_float
#     pred_pt=pred_pt.to_float
    import math
    def rad(d):
        return d * math.pi / 180.0
    lat1 = true_pt[1]
    lng1 = true_pt[0]
    lat2 = pred_pt[1]
    lng2 = pred_pt[0]
    radLat1 = rad(lat1)
    radLat2 = rad(lat2)
    a = radLat1 - radLat2
    b = rad(lng1) - rad(lng2)
    print(a)
    
    s = 2 * tf.asin(tf.sqrt(tf.pow(tf.sin(a/2),2) +
    tf.cos(radLat1)*tf.cos(radLat2)*tf.pow(tf.sin(b/2),2)))
    s = s * 6378.137
    s = round(s * 10000) / 10
    return s
model.compile(optimizer=adam, loss='mape', metrics=['accuracy', distance])
model.summary()


# 训练模型
model.fit(x_train, y_train, batch_size=64, epochs=500,validation_split=0.2)

# 评估模型
# loss, accuracy = model.evaluate(x_test, y_test)
# print('test loss: ', loss)
# print('test accuracy: ', accuracy)

# loss, accuracy = model.evaluate(x_train, y_train)
# print('train loss: ', loss)
# print('train accuracy: ', accuracy)
model.evaluate(x_test, y_test)

Model: "sequential_60"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_120 (Conv2D)         (None, 7, 4, 32)          320       
                                                                 
 max_pooling2d_120 (MaxPooli  (None, 4, 2, 32)         0         
 ng2D)                                                           
                                                                 
 conv2d_121 (Conv2D)         (None, 4, 2, 64)          51264     
                                                                 
 max_pooling2d_121 (MaxPooli  (None, 2, 1, 64)         0         
 ng2D)                                                           
                                                                 
 flatten_60 (Flatten)        (None, 128)               0         
                                                                 
 dense_124 (Dense)           (None, 1024)            

Epoch 18/500
35/35 [==============================] - 0s 8ms/step - loss: 39.6542 - accuracy: 0.0000e+00 - distance: 3653044.0000 - val_loss: 38.7302 - val_accuracy: 0.0000e+00 - val_distance: 3614826.2500
Epoch 19/500
35/35 [==============================] - 0s 8ms/step - loss: 39.4228 - accuracy: 0.0000e+00 - distance: 3663456.0000 - val_loss: 37.5171 - val_accuracy: 0.0000e+00 - val_distance: 3567292.7500
Epoch 20/500
35/35 [==============================] - 0s 8ms/step - loss: 39.5577 - accuracy: 0.0000e+00 - distance: 3677534.5000 - val_loss: 38.1604 - val_accuracy: 0.0000e+00 - val_distance: 3595759.0000
Epoch 21/500
35/35 [==============================] - 0s 8ms/step - loss: 39.5304 - accuracy: 0.0000e+00 - distance: 3684555.7500 - val_loss: 38.3629 - val_accuracy: 0.0000e+00 - val_distance: 3603768.5000
Epoch 22/500
35/35 [==============================] - 0s 8ms/step - loss: 39.3979 - accuracy: 0.0000e+00 - distance: 3643628.0000 - val_loss: 38.0930 - val_accuracy: 0.0000e+00

In [ ]:
from sklearn.preprocessing import MinMaxScaler 
# 数据预处理
path = r"../data/siping_4g.csv"


# 将MR数据构建为7*4的矩阵
data = pd.read_csv(path)

matrix = []

for index, row in data.iterrows():
    print(row) # 输出每行的索引值
    for i in range(7):
        matrix.append([i+1,
                       row['AsuLevel_' + str(i+1)],
                       row['SignalLevel_' + str(i+1)],
                       row['Dbm_' + str(i+1)]]) # 新数据集中不存在RSSI，使用Dbm(代功率的绝对值)
    print(matrix)
    break


    
    
    

def preprocess(path):
    data = pd.read_csv(path)
    # 对经纬度进行归一化的处理
    print(data.head(1))
    scaler = MinMaxScaler()
    scaler.fit(data[['Longitude', 'Latitude']].values)
    return scaler

In [ ]:
path = r"../data/siping_4g.csv"
samples, labels = load_data(path)
print(samples[0], labels[0])

In [ ]:
print(samples.shape)
print(labels.shape)

In [ ]:
# 数据变形： (3479, 7, 4)---》 (3479, 7, 4，1)
samples = samples.reshape(-1, 7,4,1) # -1自动计算

In [ ]:
from sklearn.model_selection import train_test_split

# 划分数据集
x_train, x_test, y_train, y_test = train_test_split(samples, labels, test_size=0.2)  #划分训练数据、训练标签、验证数据、验证标签

print(len(x_train))
print(len(y_train))
print(x_train[0])
print(y_train[0])

# print(x_test[100])
# print(y_test[100])

In [ ]:
# 尝试在竞争对手中的前10%，只需要多走一步

from keras.utils import np_utils # keras中numpy工具包
from keras.models import Sequential
# 二维卷积 数据池化  数据扁平化(reshape)
from keras.layers import Dense, Convolution2D, MaxPooling2D, Flatten, Dropout
# 优化器
# from keras.optimizers import Adam
from keras.optimizers import adam_v2

# 换为one-hot格式
y_train = np_utils.to_categorical(y_train, num_classes=122)
y_test = np_utils.to_categorical(y_test, num_classes=122)

# 定义顺序模型
model = Sequential()

# 第一个卷积层
# input_shape 输入数据的形状(平面)，只要在第一层进行设置
"""
    filters, 卷积核/滤波器的个数
    kernel_size, 卷积核
    strides=(1, 1),  步长
    padding='valid',  填充方式
    data_format=None,
    dilation_rate=(1, 1),
    activation=None,  激活函数
    use_bias=True,
    kernel_initializer='glorot_uniform',
    bias_initializer='zeros',
    kernel_regularizer=None,
    bias_regularizer=None,
    activity_regularizer=None,
    kernel_constraint=None,
    bias_constraint=None,
"""
model.add(
    Convolution2D(
        input_shape = (7, 4,1),
        filters = 32,
        kernel_size = 3,
        strides=1,
        padding='same',
        activation='relu'
    )
)


# 第一个池化层:不需要输入形状 28*28 --> 28*28（same填充方式）
model.add(
    MaxPooling2D(
        pool_size=2,
        strides=2,
        padding='same',
    )
)

# 第二个卷积层 池化之后 28/2  28*28--> 14*14
model.add(Convolution2D(64,5, strides=1, padding='same', activation='relu'))

# 第二个池化层  14*14
model.add(MaxPooling2D(2,2,'same'))

# 把第二个池化层的输出扁平化为一维   7*7
model.add(Flatten()) # 7*7  --> 64*7*7

# 第一个全连接层 
model.add(Dense(1024, activation='relu'))

# Dropout防止过拟合
model.add(Dropout(0.5))

# 第2个全连接层
model.add(Dense(10, activation='softmax'))


# 定义优化器  loss 交叉熵
# adam = Adam(lr=1e-4)
lr = 1e-3
adam = adam_v2.Adam(learning_rate=lr)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# 训练模型
model.fit(x_train, y_train, batch_size=64, epochs=10)

# 评估模型
loss, accuracy = model.evaluate(x_test, y_test)
print('test loss: ', loss)
print('test accuracy: ', accuracy)

loss, accuracy = model.evaluate(x_train, y_train)
print('train loss: ', loss)
print('train accuracy: ', accuracy)

In [ ]:
from sklearn.preprocessing import MinMaxScaler 
# 数据预处理
path = r"./data/siping/4g/siping_4g.csv"

# 将MR数据构建为7*4的矩阵
data = pd.read_csv(path)

matrix = []

for index, row in data.iterrows():
    print(row) # 输出每行的索引值
    for i in range(7):
        matrix.append([i+1,
                       row['AsuLevel_' + str(i+1)],
                       row['SignalLevel_' + str(i+1)],
                       row['Dbm_' + str(i+1)]]) # 新数据集中不存在RSSI，使用Dbm(代功率的绝对值)
    print(matrix)
    break


    
    
    

def preprocess(path):
    data = pd.read_csv(path)
    # 对经纬度进行归一化的处理
    print(data.head(1))
    scaler = MinMaxScaler()
    scaler.fit(data[['Longitude', 'Latitude']].values)
    return scaler

In [ ]:
import datetime
import math


def compute_time_interval(timestamp1, timestamp2):
    dateArray1 = datetime.datetime.utcfromtimestamp(timestamp1 / 1000)
    dateArray2 = datetime.datetime.utcfromtimestamp(timestamp2 / 1000)
    # otherStyleTime = dateArray1.strftime("%Y-%m-%d %H:%M:%S")
    return float((dateArray2 - dateArray1).seconds)

def distance(true_pt, pred_pt):
    lat1 = float(true_pt[1])
    lng1 = float(true_pt[0])
    lat2 = float(pred_pt[1])
    lng2 = float(pred_pt[0])
    radLat1 = rad(lat1)
    radLat2 = rad(lat2)
    a = radLat1 - radLat2
    b = rad(lng1) - rad(lng2)
    s = 2 * math.asin(math.sqrt(math.pow(math.sin(a / 2), 2) +
                                math.cos(radLat1) * math.cos(radLat2) * math.pow(math.sin(b / 2), 2)))
    s = s * 6378.137
    s = round(s * 10000) / 10
    return s

def mode_dict(path):
#     path = data_dict[drname]
    data = pd.read_csv(path)
    trajs = data.groupby(["TrajID"])
    m_d = {}
    m_d[0] = []
    m_d[1] = []
    m_d[2] = []
    for trajid, traj in trajs:
        md = list(traj['mode'])
        loc = traj[['Longitude', 'Latitude']].values
        tl = list(traj['MRTime'])
        for j in range(1, traj.shape[0]):
            if j > 0:
                delta_t = compute_time_interval(tl[j - 1], tl[j])
                delta_s = distance(loc[j - 1, :], loc[j, :])
                m = int(md[j])
                if delta_t > 0:
                    m_d[m].append(delta_s / delta_t)

    return m_d

print(mode_dict(path))